In [281]:
from gurobipy import *
import numpy as np
import math as mt

In [282]:
# Données
constraint_count=None
variable_count=None
service_count=2
project_count=4
n = 2
p=4
couts_projet = np.array([40, 50, 60, 50])

In [283]:
# Matrice d'utilte
matrice_utilite = np.array([[19, 6, 17, 2], [2, 11, 4, 18]])
print(matrice_utilite)

[[19  6 17  2]
 [ 2 11  4 18]]


In [284]:
# vecteur w
w = [2, 1]

# creer le vecteur w'
w_prime = [(w[k] - w[k+1]) for k in range(len(w)-1)]
w_prime.append(w[-1])
print(w_prime)

[1, 1]


In [285]:
# matrice utilite
u = np.array([
        [19, 6, 17, 2],
        [2, 11, 4, 18]
])
print(u)

[[19  6 17  2]
 [ 2 11  4 18]]


In [286]:
# Generation de la matrice des contraintes 

In [287]:
## les variables rk
coef_rk = []
for k in range(n):
        for i in range(n):
                coef_rk.append([])
                for j in range(n):
                        coef_rk[-1].append(0)
                coef_rk[-1][k] = 1
coef_rk = np.array(coef_rk)
print(coef_rk)


[[1 0]
 [1 0]
 [0 1]
 [0 1]]


In [288]:
## les variables bik
coef_bik = np.identity(n**2)
coef_bik *= -1
print(coef_bik)

[[-1. -0. -0. -0.]
 [-0. -1. -0. -0.]
 [-0. -0. -1. -0.]
 [-0. -0. -0. -1.]]


In [289]:
## les variables x
coef_xi = []
for k in range(n**2):
        coef_xi.append([])
        for i in range(p):
                coef_xi[-1].append(-u[k%n][i])
print(coef_xi)

[[-19, -6, -17, -2], [-2, -11, -4, -18], [-19, -6, -17, -2], [-2, -11, -4, -18]]


In [290]:
# contrainte sur le budget
budget = [0 for i in range(n*(n+1))]
budget += [couts_projet[i] for i in range(p)]
budget = np.array(budget)
print(budget)

[ 0  0  0  0  0  0 40 50 60 50]


In [291]:
# concatenation des matrices rk, bik, xk 
result_concat = np.concatenate(
        (coef_rk, np.concatenate((coef_bik, coef_xi),axis=1)),
        axis=1)

# ajout de la contrainte sur les budgets
constrainte = np.vstack([result_concat, budget])
print(constrainte)


[[  1.   0.  -1.  -0.  -0.  -0. -19.  -6. -17.  -2.]
 [  1.   0.  -0.  -1.  -0.  -0.  -2. -11.  -4. -18.]
 [  0.   1.  -0.  -0.  -1.  -0. -19.  -6. -17.  -2.]
 [  0.   1.  -0.  -0.  -0.  -1.  -2. -11.  -4. -18.]
 [  0.   0.   0.   0.   0.   0.  40.  50.  60.  50.]]


In [293]:
## second memebre
second_membre = np.array(
        [0 for i in range(n**2 + 1)]
)

In [298]:
# Coefficient de la fonction objectif
## vecteur w'
w_prime = [(w[k] - w[k+1]) for k in range(len(w)-1)]
w_prime.append(w[-1])

## coefficient fonction objectif
c = [(i+1) * w_prime[i] for i in range(n)] # coef des r_k
c += [- w_prime[i%n]  for i in range(0, n*n)] # coef des b_ik
c += [0 for i in range(p)] # ajout des coef de x
c = np.array(c)